# iAtlas Notebooks


```
Title:   How to run an iAtlas tool
Author:  David L Gibbs
Created: 2019-11-16
Purpose: Introduction to using a workflow execution service, GA4GH style
Repo: 
Notes: Does not work on google colabs.
```

# How to run an iAtlas workflow

## software install and setup

This notebook is designed to be a quick introduction to using an iAtlas workflow.

Software used: java, docker, dockstore

The dockstore client was installed using instructions from: https://dockstore.org/quick-start

This includes installing java, docker, and the dockstore client. Docker needs to be 
setup to run without sudo privileges.

The the below commands should all work:


In [1]:
!java -version

openjdk version "11.0.5" 2019-10-15
OpenJDK Runtime Environment (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04)
OpenJDK 64-Bit Server VM (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04, mixed mode, sharing)


In [6]:
!/home/dave/bin/dockstore --version
# in the terminal, just 'dockstore --version' works

Dockstore version 1.7.3
You are running the latest stable version...
If you wish to upgrade to the latest unstable version, please use the following command:
   dockstore --upgrade-unstable


In [9]:
!docker run hello-world | head -n 4


Hello from Docker!
This message shows that your installation appears to be working correctly.



In [ ]:
!cwltool --version

## Ready to launch!


Let's work with the docksore client to launch a workflow.

From the dockstore site, I used the search bar to search for 'iAtlas'.
This brought up a number of different workflows. By clicking on 
the 'ImmuneSubtypeClustering' link, we have various information pages.
The 'Launch' tab contains instructions about launching the workflow
using cwltool.

In [37]:
#!/home/dave/bin/dockstore workflow convert entry2json \
#--entry github.com/CRI-iAtlas/iatlas-workflows/ImmuneSubtypeClustering:develop > Dockstore.json
    
!/home/dave/bin/dockstore workflow convert entry2json --entry github.com/CRI-iAtlas/iatlas-workflows/Cibersort:develop > Dockstore.json
  

In [38]:
!cat Dockstore.json

{
  "input_file": {
    "path": "/tmp/fill_me_in.txt",
    "class": "File"
  },
  "cell_counts_file": {
    "path": "/tmp/fill_me_in.txt",
    "class": "File"
  },
  "output_file": "fill me in",
  "sample_name": "fill me in"
}


Now we need to fill this in with inputs, outputs, and other parameters.

Let's get some data!

In [25]:
!curl -o ebpp_test1_1to20.tsv https://raw.githubusercontent.com/CRI-iAtlas/shiny-iatlas/develop/data/ebpp_test1_1to20.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2868k  100 2868k    0     0  31.1M      0 --:--:-- --:--:-- --:--:-- 31.1M


In [28]:
!head -n 5 ebpp_test1_1to20.tsv | cut -f 1-6

GeneID	XY1	XY2	XY3	XY4	XY5
100130426	0	0	0	0	0
100133144	3.2661	2.6815	1.7301	0	0
100134869	3.9385	8.9948	6.565	1.5492	4.4709
10357	149.135	81.0777	86.4879	53.9117	66.9063


We also need the cell signatures file.

In [10]:
rundata='''
{
  "input_file": {
    "path": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_1to20.tsv",
    "class": "File"
  },
  "cell_counts_file": {
    "path": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_cell_counts.tsv",
    "class": "File"
  },
  "output_file": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_cibersort.tsv",
  "sample_name": "test1"
}
'''
fout = open('rundata.json','w')
fout.write(rundata)
fout.close()


In [11]:
!cat rundata.json


{
  "input_file": {
    "path": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_1to20.tsv",
    "class": "File"
  },
  "cell_counts_file": {
    "path": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_cell_counts.tsv",
    "class": "File"
  },
  "output_file": "/mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_cibersort.tsv",
  "sample_name": "test1"
}


In [12]:
!/home/dave/bin/dockstore workflow launch --script --entry github.com/CRI-iAtlas/iatlas-workflows/Cibersort:develop --json rundata.json

[##################################################] 100%
Created temporary working directory at '/tmp/1576875861797-0'
Successfully downloaded files for entry 'github.com/CRI-iAtlas/iatlas-workflows/Cibersort'
Creating directories for run of Dockstore launcher at: /mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/datastore//launcher-357a9e3f-cf54-4a95-a5f3-dea4e2c8b32c
Provisioning your input files to your local machine
Parsed file:///tmp/1576875861797-0/Cibersort/workflow/steps/r_tidy_utils/format_expression_file.cwl
Parsed file:///tmp/1576875861797-0/Cibersort/workflow/steps/cibersort_aggregate_celltypes/cibersort_aggregate_celltypes.cwl
Preparing download location for: #input_file from /mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/ebpp_test1_1to20.tsv into directory: /mnt/c/Users/dgibbs/Work/iAtlas/Notebooks/datastore//launcher-357a9e3f-cf54-4a95-a5f3-dea4e2c8b32c/inputs/40c7ec2e-b46f-4741-acca-c5de611c7d29
Calling out to cwltool to run your workflow
Executing: cwltool --enable-dev --non-stric

/bin/sh: 1: Syntax error: EOF in backquote substitution


And that should be about it!